## Running Queries

Here we load a random paper, search for nearest neighbors with RediSearch, and compare the title/abstract of the two papers.

In [53]:
import numpy as np

from redis.asyncio import Redis
from utils.search_index import SearchIndex
from time import time

In [25]:
redis_conn = await Redis(host='redis', port='6379')
search_index = SearchIndex('papers')

In [29]:
# Fetch one random paper from Redis to test
async def fetch_one(redis_conn):
    async for key in redis_conn.scan_iter("paper:*"):
        return await redis_conn.hgetall(key)

In [69]:
paper = await fetch_one(redis_conn)

In [70]:
paper[b'id']

b'2004.04479'

In [42]:
# Vector created in the last step!
np.frombuffer(paper[b'vector'])

array([ 3.72441314e-015,  1.15237958e-016, -1.13066037e-021,
       -3.30453649e-016, -2.91371853e-017, -3.48800595e-016,
        6.57015181e-016, -3.42299588e-019, -5.40178649e-017,
       -2.19806370e-014, -1.56736254e-011,  2.32964870e-018,
       -1.16670029e-014, -1.79329578e-014,  2.46146047e-021,
       -6.53214121e-016,  6.67494441e-012, -2.63709988e-016,
        1.53008003e-016, -1.75170009e-018, -1.47360671e-015,
       -8.57511796e-021, -1.93090158e-020,  1.60677032e-014,
        2.43855524e-011, -1.57240043e-014, -3.13561106e-021,
        1.39058087e-021, -2.14685429e-012, -2.73797807e-017,
        8.30676090e-020,  5.71511619e-022,  4.96645375e-016,
        6.02565774e-025,  4.25241043e-017,  2.73186907e-031,
        5.68803554e-021, -7.44692335e-018, -3.04318734e-015,
       -2.12282258e-016,  3.03783864e-016,  1.56030768e-018,
        5.84314423e-017, -1.01124550e-016, -1.03554774e-014,
       -1.80627261e-020,  2.77623050e-015, -1.05185623e-016,
       -4.88033530e-015,

In [77]:
# Construct search
async def search(redis_conn, search_index, k: int, paper_key: str):
    query = search_index.vector_query(
            categories = [],
            years = [],
            number_of_results = k
    )
    print("searching for nearest neighbors to", paper_key)
    start = time()
    vector = await redis_conn.hget(paper_key, "vector")
    result = await redis_conn.ft(search_index.index_name).search(
        query,
        query_params={"vec_param": vector}
    )
    print("done in", time()-start, "seconds")
    return result

In [79]:
paper_key = "paper:" + paper[b"id"].decode('utf-8')
result = await search(redis_conn, search_index, 5, paper_key)

searching for nearest neighbors to paper:2004.04479
done in 0.0042552947998046875 seconds


In [80]:
result

Result{5 total, docs: [Document {'id': 'paper:2004.04479', 'payload': None, 'vector_score': '5.36441802979e-07'}, Document {'id': 'paper:2102.11935', 'payload': None, 'vector_score': '0.294819951057'}, Document {'id': 'paper:2110.02364', 'payload': None, 'vector_score': '0.318400859833'}, Document {'id': 'paper:1704.08006', 'payload': None, 'vector_score': '0.321665585041'}, Document {'id': 'paper:1906.02896', 'payload': None, 'vector_score': '0.322323024273'}]}

In [82]:
result.docs[1]

Document {'id': 'paper:2102.11935', 'payload': None, 'vector_score': '0.294819951057'}

In [85]:
# Input document
await redis_conn.hmget(paper_key, "title", "abstract")

[b'On Adversarial Examples and Stealth Attacks in Artificial Intelligence\n  Systems',
 b"  In this work we present a formal theoretical framework for assessing and\nanalyzing two classes of malevolent action towards generic Artificial\nIntelligence (AI) systems. Our results apply to general multi-class classifiers\nthat map from an input space into a decision space, including artificial neural\nnetworks used in deep learning applications. Two classes of attacks are\nconsidered. The first class involves adversarial examples and concerns the\nintroduction of small perturbations of the input data that cause\nmisclassification. The second class, introduced here for the first time and\nnamed stealth attacks, involves small perturbations to the AI system itself.\nHere the perturbed system produces whatever output is desired by the attacker\non a specific small data set, perhaps even a single input, but performs as\nnormal on a validation set (which is unknown to the attacker). We show that 

In [84]:
# Nearest neighbor result
await redis_conn.hmget(result.docs[1].id, "title", "abstract")

[b'Non-Singular Adversarial Robustness of Neural Networks',
 b'  Adversarial robustness has become an emerging challenge for neural network\nowing to its over-sensitivity to small input perturbations. While being\ncritical, we argue that solving this singular issue alone fails to provide a\ncomprehensive robustness assessment. Even worse, the conclusions drawn from\nsingular robustness may give a false sense of overall model robustness.\nSpecifically, our findings show that adversarially trained models that are\nrobust to input perturbations are still (or even more) vulnerable to weight\nperturbations when compared to standard models. In this paper, we formalize the\nnotion of non-singular adversarial robustness for neural networks through the\nlens of joint perturbations to data inputs as well as model weights. To our\nbest knowledge, this study is the first work considering simultaneous\ninput-weight adversarial perturbations. Based on a multi-layer feed-forward\nneural network model